# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

#### <font size="1"><b> Комментарий</b></font>
<div class="alert" style="background-color:#ead7f7;color:#8737bf">
    
- Модель CatBoostClassifier лучше всего справилась с задачей, F1 на валидационнойй выборке: 0.78, F1 на тесте 0.77.
</div>

## Подготовка

### import
<hr>

In [1]:
# убераем предупреждения
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Системные библиотеки
import time
from tqdm.notebook import tqdm

In [1]:
# Основные библиотеки
import re
import numpy as np
import pandas as pd

from pymystem3 import Mystem
import transformers

from tqdm.notebook import tqdm
tqdm.pandas()

#### spacy


In [4]:
# Установка и импорт spacy
# Install spaCy (run in terminal/prompt)
import sys
!{sys.executable} -m pip install spacy
# Download spaCy's  'en' Model
!{sys.executable} -m spacy download en

import spacy
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
     |████████████████████████████████| 13.9 MB 1.4 MB/s eta 0:00:01    |█████████████████▋              | 7.6 MB 1.4 MB/s eta 0:00:05
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


#### sklearn

In [5]:
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.metrics import f1_score 
#precision_score, recall_score, accuracy_score, roc_auc_score, roc_curve

from sklearn.pipeline import Pipeline #V0

#### nltk

In [6]:
import nltk
from nltk.corpus import stopwords as nltk_stopwords

nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

count_tf_idf = TfidfVectorizer(stop_words=stopwords)

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


#### catboost

In [7]:
!pip install catboost

In [8]:
from catboost import CatBoostClassifier

### Константы

In [9]:
DATA = ['/datasets/toxic_comments.csv', 'https://code.s3.yandex.net/datasets/toxic_comments.csv']

MODEL_TOKEN_NAME = "distilbert-base-uncased" # Используемый словарь и токенизатор

RANDOM_SEED = 12345
RANDOM_STATE = 12345

#### SAMPLE

In [10]:
# Процент рассматриваемых данных
SAMPLE = 1 # количество рассматриваемых сообщений в %

#### DEVICE

DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"
DEVICE

### Функции

#### EDA первичный анализ данных 

In [11]:
def eda(data):
    print(f'Общая информация:\n{data.info()}\n\n\
shape: {data.shape}\n\n\
describe:\n{data.describe()}\n\
\nТипы данных:\n{data.dtypes}\n\
\nНаличие Nan:\n{data.isna().sum()}\n\
\nНаличие дубликатов:\n{data.duplicated().sum()}')

#### lemmatize_clear

Лемматизация и очистки сообщений
- Очистим от лишних символов

In [12]:
#
def lemmatize_clear(text):

    sentence = re.sub(r'[^a-zA-Z\']', ' ', text).lower().strip() # En

    # Parse the sentence using the loaded 'en' model object `nlp`
    doc = nlp(sentence)
    
    return " ".join([token.lemma_ for token in doc])

In [13]:
# Посмотрим как отрабатывает функция
lemmatize_clear('The striped bats are hanging on their feet for best')

'the stripe bat be hang on their foot for good'

#### balance

In [14]:
# Фукция балансировки данных по ключевому признаку
# 0 и 1 одинаковое колличество строк

def balance (data):
    
    one_featur = data[data['toxic'] == 1]
    zero_featur = data[data['toxic'] == 0]
    
    #выберем тоже коичество учайных признаков 0 что и принака 1
    zero_featur = zero_featur.sample(len(one_featur)).reset_index(drop=True)
    
    #объеденяем строки с разными признаками
    data = pd.concat([one_featur, zero_featur], ignore_index=True)
    
    #возвращаем с перемешиванием
    return data.sample(frac=1).reset_index(drop=True)

### Загрузка данных

In [15]:
# Загружаем данные
try:
    data = pd.read_csv(DATA[0], index_col=[0], parse_dates=[0])
except:
    data = pd.read_csv(DATA[1], index_col=[0], parse_dates=[0])

In [16]:
#reset index
data.reset_index(drop= True , inplace= True )

## Анализ

### Первичный анализ

In [17]:
data.shape

(159292, 2)

In [18]:
data.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


- Название столбца с ключевыми параметрами необходимо переименовать к общепринятому и как встречается в примерах моделей -> <b>'label'</b>

In [19]:
data.isna().sum()

text     0
toxic    0
dtype: int64

- Пропущеные значения отсутствуют

In [20]:
# Смотрим наличие дубликатов в строках
data['text'].duplicated().sum()

0

In [21]:
# Смотрим количество размеченых токсичных сообщений
data['toxic'].agg(['sum', 'mean','count'])

sum       16186.000000
mean          0.101612
count    159292.000000
Name: toxic, dtype: float64

- Имеется дисбаланс классов, 0 значения почти в 10 раз больше

### Продолжем анализ на сбалансированой выборки

In [22]:
# Смотрим размер сообщений
min_text = len(data['text'].min())     
max_text = len(data['text'].max())
          
print(f'''Колличество символов в сообщениях:
Минимум = {min_text}
Максимум = {max_text}
''')

Колличество символов в сообщениях:
Минимум = 5
Максимум = 4521



- Есть сообщения состояшие из 5 символов.
- После балансировки изменился максимум символов, слишком длинные сообщения редки и не размечены как таксичные, но длинные сообщения всеравно могут попадаться. Ограничение в 512 токенов оставим.

### Детальное расмотрение

In [23]:
# Посмотрим есть ли символы кроме латинских и сколько
count_ru = 0 #v1

count0, count1, count2, count3, count4 = 0, 0, 0, 0, 0
count5, count6 = 0, 0

for s in data['text']:
    
    if re.search(r'[яА-ЯёЁ]', s):
        count_ru += 1

    count0 += s.count(',')
    count1 += s.count('.') 
    count2 += s.count('?') 
    count3 += s.count('!')
    count4 += s.count('"')
    count5 += s.count("'")
    count6 += s.count("\n")
#v0
print(f'''Колличество сообщений с кириллицей: {count_ru} 
\nКолличество символов ,  {count0}
\nКолличество символов .  {count1}
\nКолличество символов ?  {count2}
\nКолличество символов !  {count3}
\nКолличество символов "  {count4}
\nКолличество символов '  {count5}
\nКолличество символов \\n  {count6}
''')
print('Всего сообщений:', len(data))

Колличество сообщений с кириллицей: 190 

Колличество символов ,  473694

Колличество символов .  678463

Колличество символов ?  71529

Колличество символов !  105396

Колличество символов "  391716

Колличество символов '  218819

Колличество символов \n  401466

Всего сообщений: 159292


In [24]:
# Посмотрим что из себя представляют сообщения с кирилическими символами
ru = 0
for s in range(len(data['text'])): #V1
    
    if re.search(r'[яА-ЯёЁ]+', data['text'][s]):
        ru += data['toxic'][s]
        
print(f'''
Колличество toxic сообщений с кириллицей: {ru.sum()}
Всего сообщений с кириллицей: {count_ru}

Соотношение: {round(ru.sum() / count_ru, 2)}
''')



Колличество toxic сообщений с кириллицей: 11
Всего сообщений с кириллицей: 190

Соотношение: 0.06



<font size="1"><b> v1 Комментарий студента</b></font>
<div class="alert" style="background-color:#ead7f7;color:#8737bf">
    
- Сообщения в основном состоят из латинских символов, и знаков препинания.
- Присутствуют незначительное колличество сообщений с кирилическими символами из них треть toxic.

In [25]:
# Считаем отношение алфавитных символов к пунктационным 
# Отношения заглавных символов к маленьким
# Отношение пробелов ко всем символам в строке

ratio_alfa, ratio_upper, ratio_lower, ratio_space = [], [], [], []

lower, upper, space = 0, 0, 0 # количество нижнего регистра, верхнего, пробелы

for _ in range(len(data['text'])):
    
    # Считаем отношения алфавитных символов к пунктуационным
    alfa = len(re.findall('\w', data['text'][_])) # печатные символы
    lower = len(re.findall('[a-z]', data['text'][_]))
    upper = len(re.findall('[A-Z]', data['text'][_]))    
    space = len(re.findall(' ', data['text'][_]))
    
    # Считаем отношения букв ко всем символам
    if alfa != 0:
        ratio_alfa.append(round(alfa / len(data['text'][_]), 2))
    else: 
        ratio_alfa.append(0)
                          
    # Считаем отношения Заглавных ко всем символам
    if (upper != 0):
        ratio_upper.append(round(upper / len(data['text'][_]), 2))
    else:
        ratio_upper.append(0)

    # Считаем отношения прописных ко всем символам
    if (lower != 0):
        ratio_lower.append(round(lower / len(data['text'][_]), 2))
    else:
        ratio_lower.append(0)

    # Считаем отношения пробелов ко всем символам
    if space != 0:
        ratio_space.append(round(space / len(data['text'][_]), 2))
    else:
        ratio_space.append(0)

In [26]:
# Добавим столбец с отношение алфавитных символов ко всему сообщению
data['ratio_alfa'] = ratio_alfa

# Добавим столбец с отношение Заглавных символов ко всему сообщению
data['ratio_upper'] = ratio_upper

# Добавим столбец с отношение Заглавных символов ко всему сообщению
data['ratio_lower'] = ratio_lower

# Добавим столбец с отношение кол. пробелов ко всему сообщению
data['ratio_space'] = ratio_space

In [27]:
data.head()

,text,toxic,ratio_alfa,ratio_upper,ratio_lower,ratio_space
0,Explanation\nWhy the edits made under my usern...,0,0.80,0.06,0.70,0.16
1,D'aww! He matches this background colour I'm s...,0,0.74,0.07,0.58,0.15
2,"Hey man, I'm really not trying to edit war. It...",0,0.80,0.02,0.78,0.18
3,"""\nMore\nI can't make any real suggestions on ...",0,0.78,0.02,0.76,0.18
4,"You, sir, are my hero. Any chance you remember...",0,0.75,0.03,0.72,0.18


In [28]:
# Посмотрим как новые признаки влияют на ключевой параметр
print(round(data.corr(), 4))

              toxic  ratio_alfa  ratio_upper  ratio_lower  ratio_space
toxic        1.0000     -0.0575       0.2151      -0.1917       0.0393
ratio_alfa  -0.0575      1.0000      -0.0352       0.4766      -0.3921
ratio_upper  0.2151     -0.0352       1.0000      -0.8282      -0.1109
ratio_lower -0.1917      0.4766      -0.8282       1.0000      -0.0021
ratio_space  0.0393     -0.3921      -0.1109      -0.0021       1.0000


In [29]:
# Посмотрим на длинные токсичные сообщения
data[(data['text'].str.len().between(1000, len(data['text'].max())))&
     (data['toxic']==1)].head()

,text,toxic,ratio_alfa,ratio_upper,ratio_lower,ratio_space
58,My Band Page's deletion. You thought I was gon...,1,0.77,0.03,0.75,0.18
641,Curse \nWikipedia has no respect for God. \nWi...,1,0.79,0.05,0.74,0.16
746,"""You know, I was thinking...\n\nNobody really ...",1,0.76,0.06,0.69,0.15
761,""" WILL MY MOTHERFUCKING SAFETY BE GUARANTEED?...",1,0.76,0.76,0.00,0.18
1165,"""\n\nSo threatening to try to have me banned f...",1,0.79,0.02,0.77,0.17


In [30]:
# Посмотрим на длинные не токсичные сообщения
data[(data['text'].str.len().between(1000, len(data['text'].max())))&(data['toxic']==0)].head()

,text,toxic,ratio_alfa,ratio_upper,ratio_lower,ratio_space
10,"""\nFair use rationale for Image:Wonju.jpg\n\nT...",0,0.79,0.02,0.77,0.17
24,"""\n\nRe-considering 1st paragraph edit?\nI don...",0,0.79,0.03,0.76,0.17
36,"""\n\n """"Mainland Asia"""" includes """"the lower b...",0,0.79,0.04,0.73,0.16
39,Notability of Rurika Kasuga\nA tag has been pl...,0,0.80,0.01,0.78,0.17
49,"""::::Jmabel; in regards to predominant scholar...",0,0.79,0.03,0.76,0.17


- В сообщениях много лишних символов, необходимо предобработать

In [31]:
# Посмотрим на короткие токсичные сообщения
data[(data['text'].str.len().between(0, 17))&
     (data['toxic']==1)]

,text,toxic,ratio_alfa,ratio_upper,ratio_lower,ratio_space
17942,ya mum \n\nfucks ya,1,0.71,0.00,0.71,0.18
60382,WHY YOU SO PIG??,1,0.69,0.69,0.00,0.19
100520,hello cow head,1,0.86,0.00,0.86,0.14
118650,FUCK YOU,1,0.88,0.88,0.00,0.12
119018,"""""""",1,0.00,0.00,0.00,0.40
120868,you suck,1,0.88,0.00,0.88,0.12
134053,YOU\n\n CUNT,1,0.41,0.41,0.00,0.47
134084,you're an asshole,1,0.82,0.00,0.82,0.12
149599,fuck you \n\nWanker,1,0.76,0.06,0.71,0.12


- имеются сообщения содержащие только """, считаю это странным. В чем причина считать это сообщение токсичным? Запомним что такое имеет место быть.

In [32]:
# Посмотрим на короткие не токсичные сообщения
data[(data['text'].str.len().between(0, 17))&(data['toxic']==0)]

,text,toxic,ratio_alfa,ratio_upper,ratio_lower,ratio_space
2499,I understand now.,0,0.82,0.06,0.76,0.12
3347,Agreed and done.,0,0.81,0.06,0.75,0.12
3631,September 2008,0,0.93,0.07,0.57,0.07
4417,And quote from it,0,0.82,0.06,0.76,0.18
8110,Achtung y'all...,0,0.69,0.06,0.62,0.06
10121,Thanks! I am sure,0,0.76,0.12,0.65,0.18
10193,"64.86.141.133""",0,0.71,0.00,0.00,0.00
12100,Re: Talk:Niihau,0,0.80,0.20,0.60,0.07
15432,"Howdy!""",0,0.71,0.14,0.57,0.00
15588,You've got mail,0,0.80,0.07,0.73,0.13


- Имется много непонятных сообщений содержащие только знаки препинания, возможно стоит провести <b>Пунктуационный разбор текста</b>. Оставим эту идею на случай необходимости улучшить результат.
- Необходимо заменить все знаки препинания кроме ` и . .
- Присутствуют команды разметки, поискать возможно есть словарь командами разметки и можно очистить от них сообщения.

### <font size="1"><b> Вывод:</b></font>
<div class="alert" style="background-color:#ead7f7;color:#8737bf">
    
1. В данных нет Nan.
2. В длинных собщениях присутствуют непристойные слова и они правельно размечены, имеется на мой взгляд ошибочное значение разметки стока содержащая только <b>"""</b> не является Toxic.
3. Всего около 10% токсичных сообщений.


4. Сообщения состоят в основном из латинских символов.
5. Присутствуют незначительное колличество сообщений с кирилическими символами.
6. Отношение алфавитных символов к пунктационным имеет слабую корелляцию с ключевым признаком
</div>

## Обработка и подготовка данных к обучению

In [33]:
# Чтобы ускорить работу берем SAMPLE случайных сообщений
df_text = data.sample(frac=SAMPLE).reset_index(drop=True)

In [34]:
# Разделяю на признаки и целевой признак
X = df_text['text']  # features
y = df_text['toxic'] # target

del df_text

In [35]:
%%time
start = time.time()
# Очистка и лемматизация выборки.
# добавил .progress_apply, поправил лемматизатор
corpus_lemma = X.progress_apply(lemmatize_clear)

t_lemma = time.time() - start

  0%|          | 0/159292 [00:00<?, ?it/s]

CPU times: user 20min 24s, sys: 14.1 s, total: 20min 38s
Wall time: 21min 15s


In [36]:
df_lemma = pd.DataFrame(corpus_lemma)

In [37]:
df_lemma.head(2)

,text
0,welcome hello and welcome to wikipedia ...
1,ungracious what of it I be not the queen ...


### Делим на выборки

In [38]:
# Добавим столбцы с новыми признаками 
df_lemma[['ratio_alfa', 'ratio_upper', 'ratio_lower', 'ratio_space']]\
= data[['ratio_alfa', 'ratio_upper', 'ratio_lower', 'ratio_space']]

df_lemma.head(3)

,text,ratio_alfa,ratio_upper,ratio_lower,ratio_space
0,welcome hello and welcome to wikipedia ...,0.80,0.06,0.70,0.16
1,ungracious what of it I be not the queen ...,0.74,0.07,0.58,0.15
2,b class no b class no b class ...,0.80,0.02,0.78,0.18


In [39]:
# Поделим на выборки, добавил valid
X_train, X_split, y_train, y_split = train_test_split(
    df_lemma, 
    y, 
    test_size= .2, #v2
    random_state = RANDOM_SEED)

X_test, X_valid, y_test, y_valid = train_test_split(
    X_split, 
    y_split, 
    test_size= .5,
    random_state = RANDOM_SEED)

In [40]:
# Смотрим как поделились выборки
print(f'''
train shape =  {X_train.shape[0]}
valid shape =  {X_valid.shape[0]}
test shape  =  {X_test.shape[0]}

summ        = {X_train.shape[0]+
               X_test.shape[0]+
               X_valid.shape[0]}

              
data        = {data.shape[0]}
''')


train shape =  127433
valid shape =  15930
test shape  =  15929

summ        = 159292

              
data        = 159292



In [41]:
# Вычислим TF-IDF для корпуса текстов
count_tf_idf = TfidfVectorizer(stop_words=stopwords)

tfidf_train = count_tf_idf.fit_transform(X_train['text'].values) # Обучаем TF-IDF

tfidf_valid = count_tf_idf.transform(X_valid['text'].values)
tfidf_test = count_tf_idf.transform(X_test['text'].values) # Трансформируем на обученой TF-IDF

<hr><hr>

## Обучение
<hr><hr>

### LogisticRegression

In [42]:
%%time
start = time.time()
pipe = Pipeline([(
    ('model',
     LogisticRegression(random_state=RANDOM_STATE, 
                        solver='saga', 
                        max_iter=10000
                        )
    ))])

param_grid = [{'model': [LogisticRegression(random_state=RANDOM_STATE, 
                                            solver='saga'
                                           )
                        ],
               'model__penalty' : ['elasticnet', 'l1', 'l2'],
               'model__C': list(range(1,10,3))
              }]
grid = GridSearchCV(pipe, 
                    param_grid=param_grid, 
                    scoring='f1', 
                    cv=6, 
                    verbose=True, 
                    n_jobs=-1
                   )

CPU times: user 293 µs, sys: 4 µs, total: 297 µs
Wall time: 303 µs


In [43]:
%%time
#V1 перенес комент
start = time.time()

best_grid = grid.fit(tfidf_valid, y_valid)

print('Лучшие параметры:', grid.best_params_)
print('Лучший результат:', grid.best_score_)

t_grid_lr = time.time() - start

Fitting 6 folds for each of 9 candidates, totalling 54 fits
Лучшие параметры: {'model': LogisticRegression(C=7, penalty='l1', random_state=12345, solver='saga'), 'model__C': 7, 'model__penalty': 'l1'}
Лучший результат: 0.7410468807837925
CPU times: user 1min 59s, sys: 378 ms, total: 1min 59s
Wall time: 1min 59s


### CatBoostClassifier

In [44]:
scores = pd.DataFrame()

model = CatBoostClassifier(iterations = 300, 
                           learning_rate = 0.5, 
                           eval_metric = 'F1', 
                           verbose = 30, 
                           random_state = RANDOM_STATE)

In [45]:
model.fit(tfidf_train, y_train)

0:	learn: 0.4760469	total: 3.41s	remaining: 16m 58s
30:	learn: 0.7095234	total: 1m 15s	remaining: 10m 55s
60:	learn: 0.7556813	total: 2m 23s	remaining: 9m 23s
90:	learn: 0.7772397	total: 3m 35s	remaining: 8m 13s
120:	learn: 0.7897559	total: 4m 46s	remaining: 7m 3s
150:	learn: 0.8019476	total: 5m 54s	remaining: 5m 49s
180:	learn: 0.8122715	total: 6m 57s	remaining: 4m 34s
210:	learn: 0.8221915	total: 8m 2s	remaining: 3m 23s
240:	learn: 0.8300378	total: 9m 7s	remaining: 2m 14s
270:	learn: 0.8337912	total: 10m 16s	remaining: 1m 5s
299:	learn: 0.8358132	total: 11m 20s	remaining: 0us


In [46]:
cbc_score = f1_score(y_valid, model.predict(tfidf_valid))
print('F1 CatBoostClassifier = ', round(cbc_score, 2))

F1 CatBoostClassifier =  0.78


#### Посмотрим как CatBoost справится с новыми фичами

In [47]:
scores = pd.DataFrame()

model_n_f = CatBoostClassifier(iterations = 1000, 
                               learning_rate = 1, 
                               eval_metric = 'F1', 
                               verbose = 200, 
                               random_state = RANDOM_STATE)

In [48]:
model_n_f.fit(X_train[['ratio_alfa', 
                       'ratio_upper', 
                       'ratio_lower', 
                       'ratio_space']],
              y_train)

0:	learn: 0.0000000	total: 26.5ms	remaining: 26.4s
200:	learn: 0.0514050	total: 5.88s	remaining: 23.4s
400:	learn: 0.0827810	total: 11.9s	remaining: 17.7s
600:	learn: 0.1083830	total: 18.1s	remaining: 12s
800:	learn: 0.1254857	total: 24s	remaining: 5.96s
999:	learn: 0.1421652	total: 30s	remaining: 0us


- Попробовал обучить CatBoost на новых фичах, результат F1 CatBoostClassifier =  0.02377
- Возможно стоит использовать как дообучение модели

In [49]:
cbc_score = f1_score(y_valid, 
                     model_n_f.predict(X_valid[['ratio_alfa', 
                                                'ratio_upper', 
                                                'ratio_lower', 
                                                'ratio_space']]))

print('F1 CatBoostClassifier = ', round(cbc_score, 5))

F1 CatBoostClassifier =  0.01011


<hr><hr>

## Вывод
<hr><hr>

### Финальный результат

Лучше всего справляется CatBoostClassifie, посмотрим на тестовых данных:

In [50]:
cbc_score_fin = f1_score(y_test, model.predict(tfidf_test))
print('F1 CatBoostClassifier = ', round(cbc_score_fin, 2))

F1 CatBoostClassifier =  0.77


- Модель CatBoostClassifier лучше всего справилась с задачей, F1 на валидационнойй выборке: 0.78, F1 на тесте 0.77.